In [71]:
import pandas as pd
import xlrd
import pyodbc
import warnings
warnings.simplefilter('ignore', UserWarning)
import sqlalchemy as sa
from datetime import datetime
import numpy as np
import duckdb
##%%
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sql_templates_base import *

## Example dataset

In [2]:
duckdb_conn = duckdb.connect(database=':memory:')
duckdb_conn.execute("""
   ATTACH 'dbname=analytics user=analytic password=R6MRbe6tT4vS2n host=192.168.9.234 port=5432'
   AS pg (TYPE POSTGRES)""")

In [5]:
create_query="""
create table transactions (
    transaction_id int,
    user_id int,
    transaction_date date,
    store_id int,
    payment_method varchar(10),
    amount float
)
;
"""

In [6]:
duckdb_conn.execute(create_query)

In [55]:
insert_query="""
insert into transactions
(transaction_id, user_id, transaction_date, store_id, payment_method, amount)
values
    (1, 1234, '2019-03-02', 1, 'cash', 5.25),
    (1, 1234, '2019-03-01', 1, 'credit', 10.75),
    (1, 1234, '2019-03-02', 2, 'cash', 25.50),
    (1, 1234, '2019-03-03', 2, 'credit', 17.00),
    (1, 4321, '2019-03-01', 2, 'cash', 20.00),
    (1, 4321, '2019-03-02', 2, 'debit', 30.00),
    (1, 4321, '2019-03-03', 1, 'cash', 3.00)
;
"""

In [56]:
insert_query=insert_query.replace("’","'")
insert_query=insert_query.replace("‘","'")

In [57]:
print(insert_query)


insert into transactions
(transaction_id, user_id, transaction_date, store_id, payment_method, amount)
values
    (1, 1234, '2019-03-02', 1, 'cash', 5.25),
    (1, 1234, '2019-03-01', 1, 'credit', 10.75),
    (1, 1234, '2019-03-02', 2, 'cash', 25.50),
    (1, 1234, '2019-03-03', 2, 'credit', 17.00),
    (1, 4321, '2019-03-01', 2, 'cash', 20.00),
    (1, 4321, '2019-03-02', 2, 'debit', 30.00),
    (1, 4321, '2019-03-03', 1, 'cash', 3.00)
;



In [58]:
duckdb_conn.execute(insert_query)

In [59]:
#  check

In [60]:
query="""
select * from transactions;
"""

In [61]:
df=duckdb_conn.execute(query).df()

In [62]:
df.head()

,transaction_id,user_id,transaction_date,store_id,payment_method,amount
0,1,1234,2019-03-02,1,cash,5.25
1,1,1234,2019-03-01,1,credit,10.75
2,1,1234,2019-03-02,2,cash,25.50
3,1,1234,2019-03-03,2,credit,17.00
4,1,4321,2019-03-01,2,cash,20.00


In [63]:
## metrics to calc

In [64]:
query="""
select
    store_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
    , avg(amount) as avg_amount
from
    transactions
group by
    store_id
order by total_amount desc
"""

In [65]:
duckdb_conn.execute(query).df()

,store_id,num_transactions,total_amount,avg_amount
0,2,4,92.5,23.125000
1,1,3,19.0,6.333333


In [66]:
_BASIC_STATS_TEMPLATE = '''
select
    {{ dim | sqlsafe }}
    , count(*) as num_transactions
    , sum(amount) as total_amount
    , avg(amount) as avg_amount
from
    transactions
group by
    {{ dim | sqlsafe }}
order by total_amount desc
'''

In [67]:
## git https://github.com/sizrailev/life-around-data-code/blob/master/pylad/sql_templates_base.py

In [68]:
def get_basic_stats_sql(dimensions):
    '''
    Returns the sql computing the number of transactions,
    as well as the total and the average transaction amounts
    for the provided list of column names as dimensions.
    '''
    # TODO: construct params
    return apply_sql_template(_BASIC_STATS_TEMPLATE, params)

## Passing a string generated outside the template

In [69]:
def get_basic_stats_sql(dimensions):
    '''
    Returns the sql computing the number of transactions,
    as well as the total and the average transaction amounts
    for the provided list of column names as dimensions.
    '''
    params = {
       'dim': '\n    , '.join(dimensions)
    }
    return apply_sql_template(_BASIC_STATS_TEMPLATE, params)

In [72]:
print(get_basic_stats_sql(['store_id', 'payment_method']))


select
    store_id
    , payment_method
    , count(*) as num_transactions
    , sum(amount) as total_amount
    , avg(amount) as avg_amount
from
    transactions
group by
    store_id
    , payment_method
order by total_amount desc


In [73]:
dimension_lists = [
    ['user_id'],
    ['store_id'],
    ['payment_method'],
    ['store_id', 'payment_method'],
]
dimension_queries = [get_basic_stats_sql(dims) for dims in dimension_lists]

In [75]:
# dimension_queries